In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
# from qutip import *

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    for idx, circ in enumerate(circuit_list): 
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        distribution1.append(dist1)
        distribution2.append(dist2)
        plt.scatter(dist1, dist2)
    plt.xlabel(name1)
    plt.ylabel(name2)
    mx = max([max(distribution1), max(distribution2)])
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return results.fits['full'].estimates['r'] 

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [8]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [57]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT

In [58]:
# make random circuits 
N_circs = 250
depth = 20

circ_list = []
for n in range(N_circs):
    circ_list.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))
circ_list = _circ.to_circuits(circ_list) 

# no need to calculate design matrices with unscented filter!
# jdict = dict()
# #hdict = dict()
# for idx, circ in enumerate(circ_list):
#     if idx%10 == 0:
#         print(idx/10, N_circs/10)
#     jdict[circ] = matrix_from_jacob(ref_model.sim.dprobs(circ), 2**circ.width)
#     #hdict[circ] = tensor_from_hessian(ref_model.sim.hprobs(circ), 2**circ.width)

---from filterpy.kalman import KalmanFilter
dog_filter = KalmanFilter(dim_x=2, dim_z=1)
print('x = ', dog_filter.x.T)
print('R = ', dog_filter.R)
print('Q = \n', dog_filter.Q)

# Unscented Kalman Filter 

In [59]:
# make a model from a model pack
MODEL_PACK = _smq1Q_XYZI
ref_model = MODEL_PACK.target_model('H+S')
filter_model = MODEL_PACK.target_model('H+S')
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01)
noise_model = noise_model.rotate(max_rotate=0.01)

In [60]:
# sample circuits
num_samples = 1000
data_set = pig.data.simulate_data(noise_model, circ_list, num_samples=num_samples)
shot_noise = 1/np.sqrt(num_samples)

In [61]:
# make rb param
rb_param = make_rb_param(noise_model, MODEL_PACK)
print(rb_param, np.sqrt(rb_param))

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 629064
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 629074
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 629084
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 629094
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 629104
0.001560381020725865 0.03950165845538469


In [62]:
# trasnition and measurement function 
def fx(x, dt):
    return x

def hx(xvec, **hxargs):
    circ = hxargs['hx_args']
    filter_model.from_vector(xvec)
    probs = filter_model.probabilities(circ)
    pvec = vector_from_outcomes(probs, hilbert_dims)
    return (num_samples*pvec + np.ones(hilbert_dims))/(num_samples + hilbert_dims)

In [63]:
# setup the filter
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints

hilbert_dims = 2**len(MODEL_PACK.processor_spec().qubit_labels)
num_params = len(filter_model.to_vector())

points = MerweScaledSigmaPoints(num_params, alpha=.1, beta=2., kappa=-1)

kf = UnscentedKalmanFilter(dim_x=num_params, dim_z=hilbert_dims, dt=1, hx=hx, fx=fx, points=points)

kf.x = np.zeros(num_params)
kf.P *= np.sqrt(rb_param)

kf.Q = 0.1*np.eye(num_params)

In [64]:
# run it
%matplotlib

plotting = True

for idx, circ in enumerate(circ_list): 
    if idx % 10 == 0 and idx != 0:
        print(idx/len(circ_list), "percent finished")
    # make observation
    counts = vector_from_outcomes(data_set[circ].counts, hilbert_dims)
    observation = (counts + np.ones(hilbert_dims))/(num_samples + hilbert_dims)

    # estimate the observation noiseetrainingcurvesforthedifferentdualandjointKalma
    shot_noise = make_multinom_covar(observation)
    R = shot_noise
     
    # update filter
    kf.predict()
    kf.update(observation, R=R, hx=hx, hx_args=circ)
    
    if(plotting):
        filter_model.from_vector(kf.x)
        naive_error = _pdftools.tvd(ref_model.probabilities(circ), data_set[circ].fractions)
        plt.scatter(idx, naive_error, c='blue', alpha=0.5)
        error = _pdftools.tvd(filter_model.probabilities(circ), data_set[circ].fractions)
        plt.scatter(idx, error, c='red', alpha=0.5)
        #plt.errorbar(idx, error, yerr=np.sqrt(np.trace(model_covar)), c='red', alpha=0.4)

Using matplotlib backend: Qt5Agg
0.04 percent finished
0.08 percent finished
0.12 percent finished
0.16 percent finished
0.2 percent finished
0.24 percent finished
0.28 percent finished
0.32 percent finished
0.36 percent finished
0.4 percent finished
0.44 percent finished
0.48 percent finished
0.52 percent finished
0.56 percent finished
0.6 percent finished
0.64 percent finished
0.68 percent finished
0.72 percent finished
0.76 percent finished
0.8 percent finished
0.84 percent finished
0.88 percent finished
0.92 percent finished
0.96 percent finished


In [54]:
# update the model
print(kf.x)
filter_model.from_vector(kf.x)

[ 0.01538269 -0.06651926  0.00362715  0.          0.          0.
  0.02507197 -0.07338504 -0.00273538  0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.12518542  0.00705204 -0.12305834  0.          0.          0.
 -0.05082502 -0.00598351  0.0457722   0.          0.          0.
 -0.04527021 -0.07912662 -0.19960582  0.          0.          0.        ]


In [56]:
# make random circuits 
N_circs = 350
depth = 20
new_circ_list = []
for n in range(N_circs):
    new_circ_list.append(random_circuit(depth, MODEL_PACK.processor_spec()))
new_circ_list = _circ.to_circuits(circ_list) 

compare_models(noise_model, ref_model, filter_model, new_circ_list, 'ref', 'filter')